# Tinyflux: Example with fictious temperature measurements

## Import Python Libraries und Settings

In [1]:
# Libraries
import pandas as pd
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta #querys welche gebraucht werden

# Settings
import warnings
warnings.filterwarnings("ignore")

# Abkürzungen für Query-Typen. Für die Übersichtlichkeit -> Ebenfalls in der Prüfung übernehmen
time = TimeQuery()
tags = TagQuery()
field = FieldQuery()

## Load function for creating a dataframe from query results into memory

In [2]:
#Kann 1:1 übernommen werden, einfach attribute anpassen wie time, sensor_id etc.

# Define the function to transform the list of Point objects into a DataFrame
def points_to_dataframe(points_list):
    data = []
    
    # Extract the relevant information
    for point in points_list:
        data.append({
            "time": point.time,  # Access time attribute
            "sensor_id": point.tags['sensor_id'],  # Access sender_id from tags dictionary
            "temperature": point.fields['temperature'],  # Access receiver_id from tags dictionary
            "status": point.tags['status'],# Access status from tags dictionary
        })

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'])

    return df

## Initialize the TinyFlux database
## (Leere temperature.db wird generiert)

In [3]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("temperatures.db")

## Importing a CSV file and show first descriptive statistics

In [4]:
# CSV-Datei in Dataframe laden
df = pd.read_csv('example_data/synthetic_temperature_readings.csv')

# Das Format des Zeitstempels in Pandas setzen
df['time']= pd.to_datetime(df['time'], format='mixed')

In [5]:
# Erste 10 Records des Dataframes anzeigen
df.head(10)

,time,sensor_id,temperature,status
0,2023-01-01 00:00:00,52,20.377761,operational
1,2023-01-01 08:44:41,93,2.217658,maintenance
2,2023-01-01 17:29:22,15,14.912385,offline
3,2023-01-02 02:14:03,72,6.517424,operational
4,2023-01-02 10:58:44,61,36.684591,operational
5,2023-01-02 19:43:25,21,-9.623282,operational
6,2023-01-03 04:28:06,83,1.266640,operational
7,2023-01-03 13:12:47,87,8.267841,operational
8,2023-01-03 21:57:28,75,14.390490,operational
9,2023-01-04 06:42:09,75,32.540876,operational


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   time         1000 non-null   datetime64[ns]
 1   sensor_id    1000 non-null   int64         
 2   temperature  1000 non-null   float64       
 3   status       1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 31.4+ KB


In [7]:
# Deskriptive Statistiken des Dataframes
df.describe(include='all') # ohne 'include = all' nur die quantitativen Angaben

,time,sensor_id,temperature,status
count,1000,1000.000000,1000.000000,1000
unique,NaN,NaN,NaN,3
top,NaN,NaN,NaN,operational
freq,NaN,NaN,NaN,858
mean,2023-07-01 23:59:59.513999872,49.560000,15.392825,NaN
min,2023-01-01 00:00:00,1.000000,-9.988124,NaN
25%,2023-04-01 23:59:59.750000128,24.000000,2.945454,NaN
50%,2023-07-01 23:59:59.500000,50.000000,16.125006,NaN
75%,2023-09-30 23:59:59.249999872,75.000000,27.474836,NaN
max,2023-12-31 00:00:00,99.000000,39.967675,NaN


## Writing a CSV file into Tinyflux / Task1 
## (temperatures.db wird mit Daten gefüllt)


In [6]:
# Read CSV and insert data into TinyFlux
with open("example_data/synthetic_temperature_readings.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"), #here is time-granularity defined, has to be as in csv file
            # example daily granularity: time = datetime.strptime(row["time"], "%Y-%m-%d")
            measurement="blockchain_transactions", #tabellenname
            fields={
                "temperature": float(row["temperature"])
                },  # Only numeric fields
            tags={
                "sensor_id": row["sensor_id"],
                "status": row["status"],  # Move non-numeric fields to tags
            }
        )
        db.insert(point)

## Writing manually into Tinyflux / Task2
## (in temperatures.db "default" data)

In [7]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc), #Granularitätsstufe Jahr, Monat, Tag, Stunden
    tags={"sensor_id": "20", "status": "operational"},
    fields={"temperature": 15.240971}
)

p2 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc),
    tags={"sensor_id": "30", "status": "maintenance"},
    fields={"temperature": 20.517200}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

2

## Querying by Time / Task3
## Greater than or equal to

In [8]:
# Definieren der Query
time_query_conditions = (time >= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) # greater than or equal to January 1, 2024, at midnight (in UTC).

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

,time,sensor_id,temperature,status
0,2024-01-01 00:00:00+00:00,20,15.240971,operational
1,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
2,2024-01-01 00:00:00+00:00,20,15.240971,operational
3,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
4,2024-01-01 00:00:00+00:00,20,15.240971,operational
5,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
6,2024-01-01 00:00:00+00:00,20,15.240971,operational
7,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
8,2024-01-01 00:00:00+00:00,20,15.240971,operational
9,2024-01-01 00:00:00+00:00,30,20.517200,maintenance


## Querying by time / Task3
## Between two specific Dates

In [ ]:
# Query Records Between Two Specific Dates/Times
start_time = datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 12, 31, 23, 59, 59, tzinfo=timezone.utc)

# Query records between two dates
time_query_conditions = (start_time <= time <= end_time)

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

## Querying by time / Task3
## For a specific day

In [10]:
# Define query conditions for a specific day (e.g., September 15, 2024)
start_time = datetime(2024, 9, 15, 0, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 9, 15, 23, 59, 59, tzinfo=timezone.utc)

# Query records between two dates
time_query_conditions = (start_time <= time <= end_time)

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

,time,sensor_id,temperature,status
0,2023-01-01 00:00:00+00:00,52,20.377761,operational
1,2023-01-01 00:00:00+00:00,52,20.377761,operational
2,2023-01-01 00:00:00+00:00,52,20.377761,operational
3,2023-01-01 00:00:00+00:00,52,20.377761,operational
4,2023-01-01 00:00:00+00:00,52,20.377761,operational
...,...,...,...,...
8007,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
8008,2024-01-01 00:00:00+00:00,20,15.240971,operational
8009,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
8010,2024-02-02 00:00:00+00:00,40,28.839000,maintenance


## Querying by time / Task3
## For a specific time

In [9]:
# Definieren der Query
time_query_conditions = (time == datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc)) 

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

,time,sensor_id,temperature,status
0,2024-01-01 00:00:00+00:00,20,15.240971,operational
1,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
2,2024-01-01 00:00:00+00:00,20,15.240971,operational
3,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
4,2024-01-01 00:00:00+00:00,20,15.240971,operational
5,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
6,2024-01-01 00:00:00+00:00,20,15.240971,operational
7,2024-01-01 00:00:00+00:00,30,20.517200,maintenance
8,2024-01-01 00:00:00+00:00,20,15.240971,operational
9,2024-01-01 00:00:00+00:00,30,20.517200,maintenance


In [ ]:
df_result_time.describe(include='all')

## Querying by Tag / Task5
## Specific Tag

In [11]:
df.status.unique() # gibt alle Werte für Tag "status" an

array(['operational', 'maintenance', 'offline'], dtype=object)

In [13]:
# Definieren der Queries
tag_query_conditions_operational = (tags.status == "operational") # abfrage nach bestimmten tags
tag_query_conditions_maintenance = (tags.status == "maintenance")
tag_query_conditions_offline = (tags.status == "offline")

# Datenabfrage mittels der definierten Queries
my_results_operational = db.search(tag_query_conditions_operational)
my_results_maintenance = db.search(tag_query_conditions_maintenance)
my_results_offline = db.search(tag_query_conditions_offline)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_tag_operational = points_to_dataframe(my_results_operational)
df_result_tag_maintenance = points_to_dataframe(my_results_maintenance)
df_result_tag_offline = points_to_dataframe(my_results_offline)

## How?

In [ ]:
temp_by_location = df.groupby(['status'])

my_results_operational = db.search(temp_by_location)

df_result_tag_operational = points_to_dataframe(my_results_operational)


### Dataframe with operational status

In [14]:
df_result_tag_operational.head(10) # Anzahl Abfragen aus dem Resultat "df_result_tag_operational"

,time,sensor_id,temperature,status
0,2023-01-01 00:00:00+00:00,52,20.377761,operational
1,2023-01-01 00:00:00+00:00,52,20.377761,operational
2,2023-01-01 00:00:00+00:00,52,20.377761,operational
3,2023-01-01 00:00:00+00:00,52,20.377761,operational
4,2023-01-01 00:00:00+00:00,52,20.377761,operational
5,2023-01-01 00:00:00+00:00,52,20.377761,operational
6,2023-01-01 00:00:00+00:00,52,20.377761,operational
7,2023-01-01 00:00:00+00:00,52,20.377761,operational
8,2023-01-02 02:14:03+00:00,72,6.517424,operational
9,2023-01-02 02:14:03+00:00,72,6.517424,operational


In [ ]:
df_result_tag_operational.describe(include='all')

### Dataframe with maintenance status

In [ ]:
df_result_tag_maintenance.head(2)

In [ ]:
df_result_tag_maintenance.describe(include='all')

### Dataframe with offline status

In [ ]:
df_result_tag_offline.head(2)

In [ ]:
df_result_tag_maintenance.describe(include='all')

In [ ]:
# Unschöne Variante
tag_query = TagQuery()
q1 = db.search(tag_query.status == "operational")

for x in q1: 
    print(x)


## Querying by Field / Task4

In [15]:
# Definieren der Queries
field_query_conditions = (field.temperature > 0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_field = points_to_dataframe(my_results)

print("\n Dataframe with temperature measurements > 0:")
df_result_field.head(2)


 Dataframe with temperature measurements > 0:


,time,sensor_id,temperature,status
0,2023-01-01 00:00:00+00:00,52,20.377761,operational
1,2023-01-01 00:00:00+00:00,52,20.377761,operational


In [ ]:
df_result_field.describe(include='all')

## Querying by Field / Task4
## total or average temperature values

In [ ]:
# Definieren der Queries / temperature grösser als 0 Grad
field_query_conditions = (field.temperature > 0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_field = points_to_dataframe(my_results)

# Calculate total and average temperature
total_temperature = df_result_field['temperature'].sum()
average_temperature = df_result_field['temperature'].mean()

# Print the results
# print("\nDataframe with temperature measurements > 0:")
# print(df_result_field.head(2))  # Display the first 2 records
print(f"\nTotal temperature: {total_temperature}")
print(f"Average temperature: {average_temperature}")

#-------------------------------------------------------------------------------------

# Definieren der Queries / measured before Mach 31 2023
time_query_conditions = (time < datetime(2023, 3, 31, 0, 0, 0, tzinfo=timezone.utc))

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result_field = points_to_dataframe(my_results)

# Calculate total and average temperature
total_temperature = df_result_field['temperature'].sum()
average_temperature = df_result_field['temperature'].mean()

# Print the results
# print("\nDataframe with temperature measurements > 0:")
# print(df_result_field.head(2))  # Display the first 2 records
print(f"\nTotal temperature: {total_temperature}")
print(f"Average temperature: {average_temperature}")

#-------------------------------------------------------------------------------------

# Give average temperature of each status-value
temp_by_location = df.groupby(['status'])['temperature'].mean()

print(temp_by_location)

## Variations of combining queries

### Querying by Time and Tag

In [ ]:
# Querying all temperatures, that were measured before Mach 31 2023 with the status "operational"

# Definieren der Queries
time_query_conditions = (time < datetime(2023, 3, 31, 0, 0, 0, tzinfo=timezone.utc))
tag_query_conditions = (tags.status == "operational")

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & tag_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_tag = points_to_dataframe(my_results)
df_result_time_tag.head(2)

In [ ]:
df_result_time_tag.describe(include='all')

### Querying by Tag and Field

In [ ]:
# Querying all temperatures higher than 20 degrees and with the status "maintenance"

# Definieren der Queries
tag_query_conditions = (tags.status == "maintenance")
field_query_conditions = (field.temperature > 20.0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(tag_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_tag_field = points_to_dataframe(my_results)
df_result_tag_field.head(2)

In [ ]:
df_result_tag_field.describe(include='all')

## Variations of queries with multiple AND and OR conditions

In [ ]:
# Querying all measurements during winter months with temperatures below 0
# OR conditions are separated with "|" (not to be confused with "/", see example below)

# Definieren der Queries
time_query_conditions = (time <= datetime(2023, 2, 28, 0, 0, 0, tzinfo=timezone.utc)) | (time >= datetime(2023, 12, 1, tzinfo=timezone.utc)) # "or" mit |
field_query_conditions = (field.temperature < 0.0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_field = points_to_dataframe(my_results)
df_result_time_field.head(2)

In [ ]:
df_result_time_field.describe(include='all')

In [ ]:
# Querying all measurements during summer months with temperatures above 0
# AND conditions are separated with "&"

# Definieren der Queries
time_query_conditions = (time <= datetime(2023, 8, 31, 0, 0, 0, tzinfo=timezone.utc)) & (time >= datetime(2023, 6, 1, tzinfo=timezone.utc))
field_query_conditions = (field.temperature > 0.0)

# Datenabfrage mittels der definierten Queries
my_results = db.search(time_query_conditions & field_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time_field_2 = points_to_dataframe(my_results)
df_result_time_field_2.head(2)

In [ ]:
df_result_time_field_2.describe(include='all')

## How Tags Are Useful for Organizing and Filtering Data in a Time-Series Database:

### Tags in a time-series database (like TinyFlux, InfluxDB, etc.) are key-value pairs that help organize and filter data more efficiently. Here's why they are important:

### - Efficient Filtering: Tags allow for fast querying and filtering of records. For example, filtering all data where location="Zurich" allows you to quickly retrieve data points related to that specific location without needing to search through the entire dataset.

### - Group and Aggregate Data: Tags can be used to group and aggregate data, as shown above. You can group records by any tag (e.g., location, device_type, status), which is useful for reporting and trend analysis. For example, analyzing the average temperature for each location or counting the number of sensor devices.

### - Improved Data Organization: In time-series databases, tags categorize data meaningfully. A sensor_id or location tag gives context to each data point, making it easier to work with complex datasets.

### - Optimized Storage: Tags are indexed efficiently, allowing for fast lookups. Unlike fields (which store raw data), tags are meant for frequent queries and indexing, making the database more efficient when handling large datasets.

### In summary, tags are crucial in a time-series database as they enable faster searches, data organization, and aggregation, making it easier to extract insights from large sets of time-series data.